# Project D - Severstal Steel Defect Detection Kaggle Competition 
## Semantic Segmentation of steel defects
### Authors: Utkrisht Rajkumar, Subrato Chakravorty, Chi-Hsin Lo

This is the demo file used to generate our figures and test our best working model.

In [ ]:
import os
import json
import gc
import cv2
import keras
from keras import backend as K
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.layers import concatenate, Input
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import multi_gpu_model
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys

## Load Model

## Test images